In [ ]:
# Importing and installing important libraries

!pip install lime
!pip install Pillow
!pip install shap
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, transform
from torch.utils.data import Dataset
import torch
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from PIL import Image,ImageOps
import torchvision
from torch import optim,nn
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import os
import tqdm
import torch.nn.functional as F
# import shap
import warnings
import sys
from google.colab import drive
drive.mount('/content/drive')
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/content/drive/MyDrive/xai/')
from util_grad import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 275 kB 5.3 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=f8461541465e900ee241c39cad5d7b50571607f40caa3f9f035a066940a21c6f
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 5.5 MB/s 
Mounted at /content/drive


In [ ]:
# Image augmentation

fields = ['f_name','tag']
with open('/content/drive/MyDrive/xai/archive/augmentation/aug_train_.csv', 'w', newline='') as file:
    writer = csv.writer(file,fieldnames = fields)
    writer.writerows(pathos)
labels = train_df['tag'].to_numpy()
filenames = train_df['f_name']
dataset = [[],[],[]]
dir = [1,2,3]
dir[0] = '/content/drive/MyDrive/xai/archive/augmentation/COVID-19_Radiography_Dataset/COVID/'
dir[1] = '/content/drive/MyDrive/xai/archive/augmentation/COVID-19_Radiography_Dataset/Normal1/'
dir[2] = '/content/drive/MyDrive/xai/archive/augmentation/COVID-19_Radiography_Dataset/Viral Pneumonia/'
pathos = []
k = 0
l = ['COVID/','Normal1/','Viral Pneumonia/']
for j in range(3):
    my_images = os.listdir(dir[j])
    for i in range(len(my_images)):
        pathos.append([])
        patho = l[j] + my_images[i]
        pathos[k].append(patho)
        pathos[k].append(j+1)
        k+=1
dir2 = ['/content/drive/MyDrive/augmented_images/covid','/content/drive/MyDrive/augmented_images/normal','/content/drive/MyDrive/augmented_images/pneumonia/']
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range=[0.2,1.0],
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest')
for k in range(3):
  total = 5250 - len(dataset[k])
  print(total)
  cnt=0
  b = total/len(dataset[k])
  b = np.ceil(b)
  print(b)
  for j in range(len(dataset[k])):
    i=0
    for batch in datagen.flow(dataset[k][j], batch_size=1,
                              save_to_dir=dir[k],
                              save_prefix='aug',
                              save_format='png'):
        cnt+=1
        i+=1
        print(cnt)
        if (i>b):

          break
    if(cnt>total):
        print(k,len(dataset[k]))
        break

PermissionError: ignored

In [ ]:
# Loading and Preparing the dataset

standard_normalization = transforms.Normalize((0.5), (0.5))
data_transforms = transforms.Compose([transforms.RandomResizedCrop(256),
                                      transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     standard_normalization])
data_transforms_1 = transforms.Compose([transforms.RandomResizedCrop(256),
                                     transforms.ToTensor(),
                                     standard_normalization])
dict_ = {1:"Covid",2:"Normal",3:"Viral Pneumonia"}

df = pd.read_csv("/content/drive/MyDrive/xai/archive/augmentation/aug_train_.csv")
df = pd.concat([df,pd.get_dummies(df.tag)], axis=1)
train, test = train_test_split(df,test_size=.2)
train_, val = train_test_split(train,test_size=.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train_ = train_.reset_index(drop=True)
val = val.reset_index(drop=True)
from torch.utils.data import  WeightedRandomSampler
class_freq = torch.tensor(train_.tag.values).bincount()
weight = 1 / class_freq
samples_weight = weight[train_.tag.values]
sampler = WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
root_dir = "/content/drive/MyDrive/xai/archive/augmentation/COVID-19_Radiography_Dataset"

trainset = Xray_dataset(train,root_dir,transform = data_transforms)
train_loader = DataLoader(trainset, batch_size=7, shuffle=True)

testset = Xray_dataset(test,root_dir,transform = data_transforms_1)
test_loader = DataLoader(testset, batch_size=32, shuffle=True)

trainset_ = Xray_dataset(train_,root_dir,transform = data_transforms)
train_loader_ = DataLoader(trainset_, batch_size=32, sampler=sampler)

valset = Xray_dataset(val,root_dir,transform = data_transforms)
val_loader = DataLoader(valset, batch_size=32, shuffle=True)

use_cuda = torch.cuda.is_available()
num_classes = 3

loaders = {"train": train_loader_,
           "test": test_loader,
           "val": val_loader}

In [ ]:
# Training and testing the model Resnet50
# On the same lines Resnet18 was also trained.

model = torchvision.models.resnet58(pretrained = True, progress = True)
model.fc = nn.Linear(in_features=2048, out_features=num_classes, bias=True)
if use_cuda:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=3e-5)
criterion = nn.CrossEntropyLoss()
train_model(20, loaders, model, optimizer, criterion,use_cuda, '/content/drive/MyDrive/sri model/resnet50_pretrained.pt')
model = torch.load('/content/drive/MyDrive/sri model/resnet50_pretrained.pt')
from sklearn.metrics import classification_report
def conf_met(model,val_loader):
    true = []
    pred = []
    test_loss = 0
    model.cuda()
    for img,tag in tqdm.tqdm(test_loader):
        img = img.cuda()
        op = model(img)
        #loss = criterion(output, target)
        #test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        pred+= list(op.data.max(1, keepdim=True)[1].cpu().squeeze().data.numpy())
        true += list(tag.data.numpy())

    print(f"\nConfusion Metrix : \n{classification_report(true,pred)}")
conf_met(model,test_loader)

In [ ]:
# Preparing the images for experimentation

dataiter = iter(train_loader)
images,label = dataiter.next()
label = label+1
images = images.cpu()

imagessvp = images.cpu().detach().numpy()
#imagessvp = np.swapaxes(imagessvp,2,3)
imagessvp = np.swapaxes(imagessvp,1,2)
imagessvp = np.swapaxes(imagessvp,2,3)
imagessvp.shape
fig,axes = plt.subplots(nrows = 12, ncols = 1, figsize=(25,75))

for ax in axes.flatten():
  ax.axis('off')
k = 0
for i in range(12):
  for j in range(1):

    axes[i].set_title(labl[k][0]+" Probability: "+str(round(probi[k][L[k][0]-1].item()*100,4))+"%"+"\n"+labl[k][1]+" Probability: "+str(round(probi[k][L[k][1]-1].item()*100,4))+"%"+"\n"+labl[k][2]+" probability: "+str(round(probi[k][L[k][2]-1].item()*100,4))+"%")
    axes[i].imshow(imagessvp[k])
    k=k+1

# fig.savefig('/content/drive/MyDrive/SRI images/SRI FINAL FINAL IMAGES/submission_original_images2')

In [ ]:
model.cpu()
l = model(images)
# l = l.detach().numpy()
sm = torch.nn.Softmax()
probi = sm(l)
l = l.cpu().detach().numpy()
L = np.argsort(-l, axis=1)
L = L+1
labl = [[]]
for i in range(12):
  for j in range(3):
    labl[i].append(dict_[int(L[i][j])])
  labl.append([])
labl.pop()


In [ ]:
# Implementing the Lime technique for normal images. Same can be done for covid and viral pneumonia images

def batch_predict(images):
    model.eval()
    batch = torch.stack(tuple(preprocess_transform(i) for i in images), dim=0)

    device = torch.device("cpu")
    model.to(device)
    batch = batch.to(device)

    logits = model(batch)
    return logits.detach().numpy()

def get_pil_transform():
  transf = transforms.Compose([
      transforms.Resize((256, 256)),
      transforms.CenterCrop(224)
  ])
  return transf

def get_preprocess_transform():
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
  transf = transforms.Compose([
      transforms.ToTensor(),
      normalize
  ])
  return transf

pill_transf = get_pil_transform()
preprocess_transform = get_preprocess_transform()

def get_image(path):
    with open(os.path.abspath(path), 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')
from lime import lime_image
explainer = lime_image.LimeImageExplainer()
k = [11,12,13,14]
i=0
fig,axes = plt.subplots(nrows = len(k), ncols = 3,figsize=(25,25))
for ax in axes.flatten():
  ax.axis('off')
for t in k:
  from skimage.segmentation import mark_boundaries
  explanation = explainer.explain_instance(imagessvp[t],
                                          batch_predict, # classification function,
                                          hide_color=0,
                                          top_labels=2,
                                          num_samples=2000)
  temp1, mask1 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
  img_boundry1 = mark_boundaries(temp1, mask1)
  temp2, mask2 = explanation.get_image_and_mask(explanation.top_labels[1], positive_only=False, num_features=5, hide_rest=False)
  img_boundry2 = mark_boundaries(temp2, mask2)

  axes[i,0].imshow(imagessvp[t])
  axes[i,0].set_title("Original Image: "+dict_[int(label[t].numpy())])
  axes[i,1].set_title("Prediction: "+labl[t][0])
  axes[i,2].set_title("Second Prediction: "+labl[t][1])
  axes[i,1].imshow(img_boundry1)
  axes[i,2].imshow(img_boundry2)
  i=i+1


In [ ]:
#  Deconvolution, Saliency, Integrated Gradients, Occlusion, Lrp techniques code for covid images. Same can be done for normal and viral pneumonia images.

!pip install captum
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF


from torchvision import models

from captum.attr import IntegratedGradients
from captum.attr import Deconvolution
from captum.attr import Saliency
from captum.attr import visualization as viz
from captum.attr import Occlusion
from captum.attr import GradientShap
from captum.attr import DeepLift


import captum.attr
model = model.cuda()
def attribute_image_features(algorithm, input,ind, **kwargs):
    model.zero_grad()
    tensor_attributions = algorithm.attribute(input,
                                              target=label[ind],
                                              **kwargs
                                             )

    return tensor_attributions


i=0

k = [0,1,2,3]
fig,axes = plt.subplots(nrows = len(k), ncols = 6,figsize=(37.5,25))
for ax in axes.flatten():
  ax.axis('off')

for ind in k:
  input = images[ind].cuda().unsqueeze(0)
  input.requires_grad = True
  imagesvp = imagessvp[ind]

  print(ind)
  saliency = Saliency(model)
  grads = saliency.attribute(input, target=label[ind]-1)
  grads = np.transpose(grads.squeeze().cpu().detach().numpy(), (1, 2, 0))
  print(ind,'sal')
  deconv = Deconvolution(model)
  gradd = deconv.attribute(input, target=label[ind]-1)
  gradd = np.transpose(gradd.squeeze(0).cpu().detach().numpy(), (1, 2, 0))
  print(ind,'dec')
  lrp = captum.attr.LRP(model)
  gradlrp = lrp.attribute(input, target=label[ind]-1)
  gradlrp = np.transpose(gradlrp.squeeze(0).cpu().detach().numpy(), (1, 2, 0))
  print(ind,'lrp')
  ig = IntegratedGradients(model)
  gradig = ig.attribute(input, target=label[ind]-1)
  gradig = np.transpose(gradig.squeeze(0).cpu().detach().numpy(), (1, 2, 0))
  print(ind,'ig')

  #gradient_shap = GradientShap(model)
  #gradshap = gradient_shap.attribute(input, baselines = input * 0,target=label[ind]-1)
  #gradshap = np.transpose(gradshap.squeeze().cpu().detach().numpy(), (1, 2, 0))
  #print(ind,'gs')
  #lime = Lime(model)
  #gradl = lime.attribute(input,target=label[ind]-1,n_samples=200)
  #gradl = np.transpose(gradl.squeeze().cpu().detach().numpy(), (1, 2, 0))
  #print(ind,'lime')

  ablator = Occlusion(model)
  #Computes occlusion attribution, ablating each 3x3 patch,
  #shifting in each direction by the default of 1.
  grado = ablator.attribute(input, target=label[ind]-1, sliding_window_shapes=(3,3,3))
  grado = np.transpose(grado.squeeze().cpu().detach().numpy(), (1, 2, 0))


  original_image = np.transpose((images[ind].cpu().detach().numpy() / 2) + 0.5, (1, 2, 0))

  a = viz.visualize_image_attr(None, original_image, plt_fig_axis=(fig,axes[i,0]),
                        method="original_image", title="Original Image, Predicted: "+labl[ind][0],use_pyplot=False)

  _ = viz.visualize_image_attr(gradd, original_image, plt_fig_axis=(fig,axes[i,1]),method="blended_heat_map",sign="absolute_value",
                              title="deconvolution",show_colorbar=True,use_pyplot=False)
  _ = viz.visualize_image_attr(grads, original_image, method="blended_heat_map",show_colorbar=True,sign="absolute_value",
                              plt_fig_axis=(fig,axes[i,2]),title="saliency",use_pyplot=False)
  _ = viz.visualize_image_attr(gradlrp, original_image, method="heat_map",show_colorbar=True,sign="all",
                              plt_fig_axis=(fig,axes[i,3]),title="Lrp",use_pyplot=False)
 # _ = viz.visualize_image_attr(gradshap, original_image, method="blended_heat_map",show_colorbar=True,sign="all",
  #                            plt_fig_axis=(fig,axes[i,4]),title="Grad Shap",use_pyplot=False)
  #_ = viz.visualize_image_attr(gradl, original_image, method="blended_heat_map",show_colorbar=True,sign="all",
   #                             plt_fig_axis=(fig,axes[i,5]),title="Grad Lime",use_pyplot=False)
  _ = viz.visualize_image_attr(grado, original_image, method="blended_heat_map",show_colorbar=True,sign="all",
                              plt_fig_axis=(fig,axes[i,4]),title="Occlusion",use_pyplot = False)
  _ = viz.visualize_image_attr(gradig, original_image, method="blended_heat_map",show_colorbar=True,sign="all",
                              plt_fig_axis=(fig,axes[i,5]),title="Integrated Gradients",use_pyplot = False)
  i = i+1

In [ ]:
# Gradcam++ code for covid images.
# Same can be done for AblationCam and Xgradcam by changing the fucntions.

# !pip install grad-cam
# !pip install ttach
# from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM
# from pytorch_grad_cam.utils.image import show_cam_on_image
i=0
k = [11,12,13,14]
fig,axes = plt.subplots(nrows = len(k), ncols = 4,figsize = (25,25))
for ax in axes.flatten():
    ax.axis('off')
i=0
for ind in k:
  imagevp = imagessvp[ind]
  norm = (imagevp - np.min(imagevp)) / (np.max(imagevp) - np.min(imagevp))
  target_layer = model.layer4[-1]
  cam = GradCAMPlusPlus(model=model.cpu(), target_layer=target_layer)
  grayscale_cam = cam(images[ind].unsqueeze(0), target_category=0)
  grayscal_cam = grayscale_cam[0,:]
  visualization1 = show_cam_on_image(norm, grayscal_cam)


  cam = GradCAMPlusPlus(model=model, target_layer=target_layer)
  grayscale_cam = cam(images[ind].unsqueeze(0), target_category=1)
  grayscal_cam = grayscale_cam[0,:]
  visualization2 = show_cam_on_image(norm, grayscal_cam)

  cam = GradCAMPlusPlus(model=model, target_layer=target_layer)
  grayscale_cam = cam(images[ind].unsqueeze(0), target_category=2)
  grayscal_cam = grayscale_cam[0,:]
  visualization3 = show_cam_on_image(norm, grayscal_cam)



  vis = [visualization1,visualization2,visualization3]
  tit = ['Gradcam++ Covid Areas','Gradcam++ Normal Areas','Gradcam++ Viral Pneumonia Areas']
  axes[i,0].set_title('Original Image, Predicted: '+labl[ind][0])
  original_image = np.transpose((images[ind].cpu().detach().numpy() / 2) + 0.5, (1, 2, 0))
  axes[i,0].imshow(original_image)
  q=1
  for d in L[ind]:
    axes[i,q].set_title(tit[d-1])
    axes[i,q].imshow(vis[d-1])
    q=q+1
  i=i+1